<a href="https://colab.research.google.com/github/ALXAVIER-DEV/Spark/blob/master/Pyspark_hands_on.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
## Instalando o java
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [ ]:
## Baixando o spark
!wget -q https://dlcdn.apache.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz

In [ ]:
##Unzip spark
!tar xf /content/spark-3.2.1-bin-hadoop3.2.tgz

In [ ]:
## Criando variaveis de ambiente necessarias
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop3.2"

In [ ]:
## Atalho : Instalar a lib find spark no sistema e import'a-lo como uma lib regular
!pip install -q findspark

In [ ]:
## Importando a lib
import  findspark
findspark.init()

In [ ]:
#Criando a session
from pyspark.sql import SparkSession
spark = SparkSession.builder\
        .master('local')\
        .appName('spark_handson')\
        .getOrCreate()

In [ ]:
#importando o dataset
from google.colab import files
arquivo = files.upload()

In [ ]:
##Criar Dataframe
df = spark.read.csv('imdb_top_1000.csv', header=True, inferSchema=True)

In [ ]:

df.printSchema()

In [ ]:
df.show()

In [ ]:
df_movies = df.select('Series_Title','Released_Year','Runtime','Genre','Director','No_of_Votes','Star1','Star2','Star3','Star4')

In [ ]:
df_movies.show()

In [ ]:
df_vote = df_movies.withColumn('votos',df_movies['No_of_Votes'].cast('int')).drop('No_of_Votes')

In [ ]:
df_vote.filter(df_vote.votos > 1000000).show()

In [ ]:
from pyspark.sql.functions import max
df_max = df_vote.agg(max('votos').alias('max_votos')).show()

In [ ]:
df_max_votos = df_vote.filter(df_vote.votos == 2343110 )
df_max_votos.show()

In [ ]:
from pyspark.sql.functions import  count, col, asc, desc
df_sum = df_vote.groupBy('Genre').count()
df_sum.show(truncate=False)

In [ ]:
df_sum.orderBy(col('count').desc()).show()

In [ ]:
df_vote.select('Series_Title','Released_Year','Director').filter(df_vote.Genre.isin('Comedy', 'Action', 'Drama')).show()

In [ ]:
df_vote.select('Series_Title','Released_Year','Director').filter(~df_vote.Genre.isin('Comedy', 'Action', 'Drama')).show()

In [ ]:
### Use SQL
df_vote.createOrReplaceTempView('movies')
spark.sql('SELECT Released_Year, count(*) as qtd FROM movies GROUP BY Released_Year ORDER BY qtd desc').show(truncate=False)